In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import re
import os
import requests

def download_video(url, save_directory):
    # Create the save directory if it doesn't exist
    os.makedirs(save_directory, exist_ok=True)

    # Get the filename from the URL
    filename = url.split('?')[0].split('/')[-1]

    # Path to save the video
    save_path = os.path.join(save_directory, filename)

    # Send a GET request to download the video
    response = requests.get(url, stream=True)

    # Check if the request was successful
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)
        print("Video downloaded successfully!")
    else:
        print("Failed to download the video.")

def Video_url_getter(url):
    # Set up the Chrome driver
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver_service = Service('path_to_chromedriver')  # Replace with the path to your chromedriver executable
    driver = webdriver.Chrome(service=driver_service, options=chrome_options)
    
    try:
        # Open the web page
        driver.get(url)

        # Locate the button element
        button = driver.find_element(By.XPATH, '//*[@id="maincontent"]/div/div/div/div/div[2]/div[1]/div[4]/div/div[2]/button')

        # Perform the desired action on the button
        button.click()

        # Wait for the page to load completely
        wait = WebDriverWait(driver, 10)
        # Add appropriate wait conditions if needed

        # Retrieve the response or perform further actions
        current_url = driver.current_url
        page_source = driver.page_source

        # Parse the page source using BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')

        # Find all the links in the page
        links = soup.find_all('a')

        # Extract URLs starting with "https://download"
        download_urls = []
        for link in links:
            url = link.get('href')
            if url and url.startswith('https://download'):
                download_urls.append(url)

        return download_urls

    finally:
        driver.quit()

In [2]:
def select_urls_with_mp4_substring(urls):
    # Select URLs with ".mp4" as a substring
    mp4_urls = [url for url in urls if ".mp4" in url]

    return mp4_urls

In [2]:
def load_urls_from_file(file_path):
    with open(file_path, 'r') as file:
        urls = [line.strip() for line in file.readlines()]
    return urls

url_list = load_urls_from_file("relative_path_to_file_containing_urls")

In [3]:
len(url_list)

10

In [6]:
for i in range(len(url_list)):
    url = url_list[i]
    # print(url)
    video_urls = Video_url_getter(url)
    mp4_urls = select_urls_with_mp4_substring(video_urls)
    video_urls = list(set(mp4_urls))
    # print(mp4_urls)
    print(video_urls)
    download_video(mp4_urls[0], "path_to_folder_where_video_is_to_be_downloaded"+str(i))

['https://download.ted.com/products/166342.mp4?apikey=acme-roadrunner']
Video downloaded successfully!
['https://download.ted.com/products/166267.mp4?apikey=acme-roadrunner']
Video downloaded successfully!
['https://download.ted.com/products/166152.mp4?apikey=acme-roadrunner']
Video downloaded successfully!
['https://download.ted.com/products/165004.mp4?apikey=acme-roadrunner']
Video downloaded successfully!
['https://download.ted.com/products/165009.mp4?apikey=acme-roadrunner']
Video downloaded successfully!
['https://download.ted.com/products/164878.mp4?apikey=acme-roadrunner']
Video downloaded successfully!
['https://download.ted.com/products/163647.mp4?apikey=acme-roadrunner']
Video downloaded successfully!
['https://download.ted.com/products/163005.mp4?apikey=acme-roadrunner']
Video downloaded successfully!
['https://download.ted.com/products/161620.mp4?apikey=acme-roadrunner']
Video downloaded successfully!
['https://download.ted.com/products/157330.mp4?apikey=acme-roadrunner']
V

In [7]:
import os
import moviepy.editor as mp

for i in range(len(url_list)):
    path = "path_to_folder_where_video_has_been_downloaded"+str(i)
    os.chdir(path)
    for file in os.listdir():
        if file.endswith(".mp4"):
            video = mp.VideoFileClip(file)
            audio = video.audio
            audio.write_audiofile(file[:-3]+"wav")

MoviePy - Writing audio in 166342.wav


MoviePy - Done.
MoviePy - Writing audio in 166267.wav


MoviePy - Done.
MoviePy - Writing audio in 166152.wav


MoviePy - Done.
MoviePy - Writing audio in 165004.wav


MoviePy - Done.
MoviePy - Writing audio in 165009.wav


...

MoviePy - Done.
MoviePy - Writing audio in 163647.wav


MoviePy - Done.
MoviePy - Writing audio in 163005.wav


MoviePy - Done.
MoviePy - Writing audio in 161620.wav


MoviePy - Done.
MoviePy - Writing audio in 157330.wav


MoviePy - Done.


In [4]:
import os

for i in range(len(url_list)):
    path = "path_to_folder_where_video_has_been_downloaded"+str(i)
    os.chdir(path)
    for file in os.listdir():
        if file.endswith(".mp4"):
            os.remove(file)